# Домашнее задание 3

В этом задании напишем простое решение классификации датасета `FashionMNIST`, а затем будем его улучшать с помощью:
- dropout;
- batch normalization;
- LR scheduler;

В конце сохраним модель в файл и убедимся, что этот файл можем в дальнейшем прочитать.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from dataclasses import dataclass
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(torch.cuda.is_available())

True


In [4]:
torch.manual_seed(987)

In [5]:
train_dataset = FashionMNIST(
    root="./data", train=True, download=True, transform=ToTensor()
)
test_dataset = FashionMNIST(
    root="./data", train=False, download=True, transform=ToTensor()
)

In [6]:
X_train = train_dataset.data.float().to(device)
y_train = train_dataset.targets.to(device)
X_test = test_dataset.data.float().to(device)
y_test = test_dataset.targets.to(device)

In [88]:
@dataclass
class TrainConfig:
    lr: float = 1e-1
    total_iterations: int = 100


# Для оценки будем использовать метрику accuracy
# Подумайте (опционально), какие еще метрики можно использовать
def calculate_accuracy(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_true).float().sum()
    accuracy = correct / y_true.shape[0]
    return accuracy.item()

## Задание №1

Попробуйте реализовать простой бейзлайн с несколькими слоями:
- Linear
- ReLU
- Linear

Вставьте свою релизацию `SimpleModel` в проверку.
Вам нужно дописать и сдать как `SimpleModel`, так и `train_loop`.

Используйте кросс-энтропию как функцию потерь.

In [8]:
X_train.shape

torch.Size([60000, 28, 28])

In [9]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, num_classes: int, input_dim: int = 28 * 28):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.view(x.shape[0], -1)  # Преобразуем изображение в вектор
        return self.net(x)  # Пропускаем через `self.net`

In [89]:
def train_loop(model, X_train, y_train, X_val, y_val, config):
    """Обучает модель, логирует потери и метрики."""
    
    # Оптимизатор и функция потерь
    optimizer = optim.SGD(model.parameters(), lr=config.lr)
    loss_fn = nn.CrossEntropyLoss()
    
    best_accuracy = 0.0

    for epoch in range(config.total_iterations):
        model.train()  # Включаем режим обучения

        # Прямой проход
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)

        # Обратное распространение ошибки
        optimizer.zero_grad()  # Обнуляем градиенты
        loss.backward()  # Вычисляем градиенты
        optimizer.step()  # Обновляем параметры

        # Логирование
        # if epoch % 10 == 0:
        model.eval()
        test_Loss = calculate_accuracy(model(X_val), y_val)
        print((f"Epoch {epoch}, accuracy = {test_Loss}"))
            # model.eval()  # Выключаем dropout/batchnorm
            # with torch.no_grad():
            #     y_val_pred = model(X_val)
            #     val_loss = loss_fn(y_val_pred, y_val)

            #     accuracy = (y_val_pred.argmax(dim=1) == y_val).float().mean().item()
            #     print(f"Epoch {epoch}: Train Loss = {loss.item():.4f}, Val Loss = {val_loss.item():.4f}, Accuracy = {accuracy:.4f}")
        if test_Loss > best_accuracy:
            best_accuracy = test_Loss
            torch.save(model.state_dict(), "best_model.pth")

## Задание №2
Какое максимальное значение метрики accuracy удалось получить в процессе обучения?

Округлите до 3 значений после запятой


In [40]:
model = SimpleModel(input_dim=28*28, num_classes=10).to(device)
print(model)


SimpleModel(
  (net): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [41]:
config = TrainConfig()

In [44]:
train_loop(model, X_train, y_train, X_test, y_test, config)

Epoch 0, accuracy = 0.7630999684333801
Epoch 1, accuracy = 0.7567999958992004
Epoch 2, accuracy = 0.7642999887466431
Epoch 3, accuracy = 0.7579999566078186
Epoch 4, accuracy = 0.7651999592781067
Epoch 5, accuracy = 0.7584999799728394
Epoch 6, accuracy = 0.7655999660491943
Epoch 7, accuracy = 0.758899986743927
Epoch 8, accuracy = 0.7664999961853027
Epoch 9, accuracy = 0.7599999904632568
Epoch 10, accuracy = 0.7671999931335449
Epoch 11, accuracy = 0.7616999745368958
Epoch 12, accuracy = 0.7683999538421631
Epoch 13, accuracy = 0.762499988079071
Epoch 14, accuracy = 0.7696999907493591
Epoch 15, accuracy = 0.762999951839447
Epoch 16, accuracy = 0.7710999846458435
Epoch 17, accuracy = 0.763700008392334
Epoch 18, accuracy = 0.7717999815940857
Epoch 19, accuracy = 0.7644999623298645
Epoch 20, accuracy = 0.7721999883651733
Epoch 21, accuracy = 0.765500009059906
Epoch 22, accuracy = 0.7734999656677246
Epoch 23, accuracy = 0.7666999697685242
Epoch 24, accuracy = 0.7745999693870544
Epoch 25, accur

## Задание №3
Добавьте один `dropout` слой в вашу модель.

_Подумайте, что может поменяться при перестановке ReLU и Dropout слоев местами._

In [56]:
class DropoutModel(nn.Module):
    def __init__(self, input_dim=784, num_classes=10, dropout_rate=0.2):
        super().__init__()
        hidden_dim=512
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),  # Dropout после ReLU
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Преобразуем изображение в вектор
        return self.net(x)  # Пропускаем через `self.net`

In [50]:
class DropoutBeforeReluModel(nn.Module):
    def __init__(self, input_dim=784, num_classes=10, dropout_p=0.2):
        super().__init__()
        hidden_dim=512
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Dropout(p=dropout_p),  # Dropout перед ReLU
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Преобразуем изображение в вектор
        return self.net(x)  # Пропускаем через `self.net`

In [57]:
# Модель после слоя Relu
dropout_model = DropoutModel(input_dim=28*28, num_classes=10).to(device)
print(dropout_model)
config = TrainConfig()

DropoutModel(
  (net): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [58]:
train_loop(dropout_model, X_train, y_train, X_test, y_test, config)

Epoch 0, accuracy = 0.16099999845027924
Epoch 1, accuracy = 0.2272999882698059
Epoch 2, accuracy = 0.3061999976634979
Epoch 3, accuracy = 0.2522999942302704
Epoch 4, accuracy = 0.4472000002861023
Epoch 5, accuracy = 0.5800999999046326
Epoch 6, accuracy = 0.6317999958992004
Epoch 7, accuracy = 0.6502999663352966
Epoch 8, accuracy = 0.6635000109672546
Epoch 9, accuracy = 0.67249995470047
Epoch 10, accuracy = 0.6780999898910522
Epoch 11, accuracy = 0.6832000017166138
Epoch 12, accuracy = 0.6894999742507935
Epoch 13, accuracy = 0.6943999528884888
Epoch 14, accuracy = 0.6977999806404114
Epoch 15, accuracy = 0.6995999813079834
Epoch 16, accuracy = 0.7037000060081482
Epoch 17, accuracy = 0.707099974155426
Epoch 18, accuracy = 0.7093999981880188
Epoch 19, accuracy = 0.7107999920845032
Epoch 20, accuracy = 0.7127000093460083
Epoch 21, accuracy = 0.7152000069618225
Epoch 22, accuracy = 0.7160999774932861
Epoch 23, accuracy = 0.7184000015258789
Epoch 24, accuracy = 0.7206999659538269
Epoch 25, ac

In [53]:
# Модель перед слоем Relu
dropout_before_relu_model = DropoutBeforeReluModel(input_dim=28*28, num_classes=10).to(device)
print(dropout_before_relu_model)
config = TrainConfig()

DropoutBeforeReluModel(
  (net): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [54]:
train_loop(dropout_before_relu_model, X_train, y_train, X_test, y_test, config)

Epoch 0, accuracy = 0.18439999222755432
Epoch 1, accuracy = 0.14479999244213104
Epoch 2, accuracy = 0.3456999957561493
Epoch 3, accuracy = 0.32839998602867126
Epoch 4, accuracy = 0.44759997725486755
Epoch 5, accuracy = 0.5145999789237976
Epoch 6, accuracy = 0.5586999654769897
Epoch 7, accuracy = 0.5769999623298645
Epoch 8, accuracy = 0.5900999903678894
Epoch 9, accuracy = 0.6028000116348267
Epoch 10, accuracy = 0.6168999671936035
Epoch 11, accuracy = 0.6200999617576599
Epoch 12, accuracy = 0.6180999875068665
Epoch 13, accuracy = 0.6315000057220459
Epoch 14, accuracy = 0.6405999660491943
Epoch 15, accuracy = 0.6340999603271484
Epoch 16, accuracy = 0.6450999975204468
Epoch 17, accuracy = 0.6481999754905701
Epoch 18, accuracy = 0.6548999547958374
Epoch 19, accuracy = 0.6536999940872192
Epoch 20, accuracy = 0.6572999954223633
Epoch 21, accuracy = 0.6615999937057495
Epoch 22, accuracy = 0.6624999642372131
Epoch 23, accuracy = 0.6640999913215637
Epoch 24, accuracy = 0.6660000085830688
Epoch 

## Задание №4
Какое максимальное значение accuracy получилось в ходе обучения модели? 

Округлите до 3х знаков после запятой и отправьте в ЛМС.

In [ ]:
train_loop(dropout_model, X_train, y_train, X_test, y_test, config)

## Задание №5

Добавьте `BatchNorm` в вашу модель.
Отправьте в ЛМС реализацию.

Стоит ли делать BatchNorm до ReLU или после него?
Это дискуссионный вопрос, чаще всего применяют сначала нелинейность, затем Batch Norm.
Один из аргументов: при таком подходе данные на выходе будут иметь среднее 0 - что и ожидают люди, когда добавляют нормализацию.

_[Дискуссия на Reddit](https://www.reddit.com/r/MachineLearning/comments/67gonq/d_batch_normalization_before_or_after_relu/)_

Для определенности в этом задании будем следовать такому порядку: сначала ReLU, затем Batch Norm.

In [62]:
class BatchNormModel(nn.Module):
    def __init__(self, input_dim=784,  num_classes=10, dropout_p=0.2):
        hidden_dim=512
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),  # BatchNorm после ReLU
            nn.Dropout(p=dropout_p),
            nn.Linear(hidden_dim, num_classes)
        )
    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Преобразуем изображение в вектор
        return self.net(x)  # Пропускаем через `self.net`

In [63]:
# Модель после слоя Relu
batch_norm_model = BatchNormModel(input_dim=28*28, num_classes=10).to(device)
print(batch_norm_model)
config = TrainConfig()

BatchNormModel(
  (net): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Задание №6
Какое максимальное значение `accuracy` получилось в ходе обучения модели? 

Округлите до 3х знаков после запятой.

In [64]:
train_loop(batch_norm_model, X_train, y_train, X_test, y_test, config)

Epoch 0, accuracy = 0.050999999046325684
Epoch 1, accuracy = 0.06879999488592148
Epoch 2, accuracy = 0.08859999477863312
Epoch 3, accuracy = 0.11009999364614487
Epoch 4, accuracy = 0.13830000162124634
Epoch 5, accuracy = 0.16679999232292175
Epoch 6, accuracy = 0.19329999387264252
Epoch 7, accuracy = 0.21779999136924744
Epoch 8, accuracy = 0.24149999022483826
Epoch 9, accuracy = 0.2622999846935272
Epoch 10, accuracy = 0.28299999237060547
Epoch 11, accuracy = 0.30799999833106995
Epoch 12, accuracy = 0.32690000534057617
Epoch 13, accuracy = 0.3479999899864197
Epoch 14, accuracy = 0.3666999936103821
Epoch 15, accuracy = 0.3837999999523163
Epoch 16, accuracy = 0.3985999822616577
Epoch 17, accuracy = 0.41290000081062317
Epoch 18, accuracy = 0.42879998683929443
Epoch 19, accuracy = 0.4415999948978424
Epoch 20, accuracy = 0.45509999990463257
Epoch 21, accuracy = 0.46629998087882996
Epoch 22, accuracy = 0.47609999775886536
Epoch 23, accuracy = 0.48649999499320984
Epoch 24, accuracy = 0.49480000

Результат batch normalization мог не особо порадовать.
Но не спешите с выводами насчет этого слоя!

Попробуйте обучить заново все три модели со значением `lr=1e-2` (в 10 раз больше).
Сравните результаты моделей и сделайте вывод.

## Задание №7
Добавьте `LRscheduler` в вашу модель.

Подробнее про `schedulers` можно почитать в [документации](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)

In [13]:
from torch.optim.lr_scheduler import StepLR


def train_loop_with_scheduler(
    model,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    ...
    scheduler = StepLR(..., step_size=5, gamma=0.1)
    ...

In [70]:
from torch.optim.lr_scheduler import StepLR
def train_loop_with_scheduler(model, X_train, y_train, X_val, y_val, config):
    """Обучает модель, логирует потери и метрики."""
    
    # Оптимизатор и функция потерь
    optimizer = optim.SGD(model.parameters(), lr=config.lr)
    loss_fn = nn.CrossEntropyLoss()

    scheduler = StepLR(optimizer, step_size=10, gamma=0.01)

    for epoch in range(config.total_iterations):
        model.train()  # Включаем режим обучения

        # Прямой проход
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)

        # Обратное распространение ошибки
        optimizer.zero_grad()  # Обнуляем градиенты
        loss.backward()  # Вычисляем градиенты
        optimizer.step()  # Обновляем параметры
        
        scheduler.step()
        # Логирование
        # if epoch % 10 == 0:
        model.eval()
        test_Loss = calculate_accuracy(model(X_val), y_val)
        print((f"Epoch {epoch}, accuracy = {test_Loss}"))
            # model.eval()  # Выключаем dropout/batchnorm
            # with torch.no_grad():
            #     y_val_pred = model(X_val)
            #     val_loss = loss_fn(y_val_pred, y_val)

            #     accuracy = (y_val_pred.argmax(dim=1) == y_val).float().mean().item()
            #     print(f"Epoch {epoch}: Train Loss = {loss.item():.4f}, Val Loss = {val_loss.item():.4f}, Accuracy = {accuracy:.4f}")

In [ ]:
batch_norm_model = BatchNormModel(input_dim=28*28, num_classes=10).to(device)
print(batch_norm_model)
config = TrainConfig()

BatchNormModel(
  (net): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
train_loop(batch_norm_model, X_train, y_train, X_test, y_test, config)

Epoch 0, accuracy = 0.24539999663829803
Epoch 1, accuracy = 0.2712000012397766
Epoch 2, accuracy = 0.29729998111724854
Epoch 3, accuracy = 0.32330000400543213
Epoch 4, accuracy = 0.3449999988079071
Epoch 5, accuracy = 0.36579999327659607
Epoch 6, accuracy = 0.3837999999523163
Epoch 7, accuracy = 0.4017999768257141
Epoch 8, accuracy = 0.420199990272522
Epoch 9, accuracy = 0.4348999857902527
Epoch 10, accuracy = 0.4486999809741974
Epoch 11, accuracy = 0.46209999918937683
Epoch 12, accuracy = 0.47669997811317444
Epoch 13, accuracy = 0.48969998955726624
Epoch 14, accuracy = 0.49879997968673706
Epoch 15, accuracy = 0.5094000101089478
Epoch 16, accuracy = 0.5187999606132507
Epoch 17, accuracy = 0.5282999873161316
Epoch 18, accuracy = 0.5358999967575073
Epoch 19, accuracy = 0.5426999926567078
Epoch 20, accuracy = 0.5497999787330627
Epoch 21, accuracy = 0.5557999610900879
Epoch 22, accuracy = 0.561199963092804
Epoch 23, accuracy = 0.56659996509552
Epoch 24, accuracy = 0.5704999566078186
Epoch 

In [72]:
train_loop_with_scheduler(batch_norm_model, X_train, y_train, X_test, y_test, config)

Epoch 0, accuracy = 0.040300000458955765
Epoch 1, accuracy = 0.052799999713897705
Epoch 2, accuracy = 0.06390000134706497
Epoch 3, accuracy = 0.07689999788999557
Epoch 4, accuracy = 0.09319999814033508
Epoch 5, accuracy = 0.11089999973773956
Epoch 6, accuracy = 0.12879998981952667
Epoch 7, accuracy = 0.148499995470047
Epoch 8, accuracy = 0.1712999939918518
Epoch 9, accuracy = 0.19699999690055847
Epoch 10, accuracy = 0.1996999979019165
Epoch 11, accuracy = 0.20149999856948853
Epoch 12, accuracy = 0.2043999880552292
Epoch 13, accuracy = 0.20549999177455902
Epoch 14, accuracy = 0.20749999582767487
Epoch 15, accuracy = 0.20979999005794525
Epoch 16, accuracy = 0.210999995470047
Epoch 17, accuracy = 0.21169999241828918
Epoch 18, accuracy = 0.21310000121593475
Epoch 19, accuracy = 0.21449999511241913
Epoch 20, accuracy = 0.2150999903678894
Epoch 21, accuracy = 0.21559999883174896
Epoch 22, accuracy = 0.2158999890089035
Epoch 23, accuracy = 0.2157999873161316
Epoch 24, accuracy = 0.21689999103

KeyboardInterrupt: 

## Задание №8

Поэксперементируйте с параметрами нейронной сети, попробуйте добиться максимальной метрики `accuracy`.

- попробуйте комбинацию Drouput + Batch Normalization и подумайте, как лучше всего раскрыть силу batch normalization (вспомните эксперименты с lr);
- попробуйте подвигать вероятность в Dropout;
- ну, или подержите обучение подольше, поставив больше шагов :)

В ЛМС нужно сдать код класса `ExpModel`.
Вам необходимо выбить accuracy > 80%, чтобы сдать этот пункт.

In [27]:
torch.manual_seed(987)


# Ваш код модели и ее обучения при seed = 987
class ExpModel: ...


model = ExpModel()
config = TrainConfig(...)
train_loop(model, X_train, y_train, X_test, y_test, config)

In [90]:
class ExpModel(nn.Module):
    def __init__(self, input_dim=784, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.2),
            
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Преобразуем изображение в вектор
        return self.net(x)  # Пропускаем через `self.net`

In [91]:
exp_model = ExpModel(input_dim=28*28, num_classes=10).to(device)
print(exp_model)
config = TrainConfig()

ExpModel(
  (net): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [92]:
train_loop(exp_model, X_train, y_train, X_test, y_test, config)

Epoch 0, accuracy = 0.5778999924659729
Epoch 1, accuracy = 0.6714999675750732
Epoch 2, accuracy = 0.6633999943733215
Epoch 3, accuracy = 0.7511999607086182
Epoch 4, accuracy = 0.6873999834060669
Epoch 5, accuracy = 0.7305999994277954
Epoch 6, accuracy = 0.7099999785423279
Epoch 7, accuracy = 0.762499988079071
Epoch 8, accuracy = 0.7490999698638916
Epoch 9, accuracy = 0.8001999855041504
Epoch 10, accuracy = 0.7795999646186829
Epoch 11, accuracy = 0.8154000043869019
Epoch 12, accuracy = 0.7975999712944031
Epoch 13, accuracy = 0.8230999708175659
Epoch 14, accuracy = 0.8047999739646912
Epoch 15, accuracy = 0.8241999745368958
Epoch 16, accuracy = 0.8069999814033508
Epoch 17, accuracy = 0.8233000040054321
Epoch 18, accuracy = 0.8070999979972839
Epoch 19, accuracy = 0.8183000087738037
Epoch 20, accuracy = 0.8087999820709229
Epoch 21, accuracy = 0.8167999982833862
Epoch 22, accuracy = 0.8127999901771545
Epoch 23, accuracy = 0.8201999664306641
Epoch 24, accuracy = 0.8204999566078186
Epoch 25, a

In [87]:
model.load_state_dict(torch.load("best_model.pth"))

FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pth'

Наконец, сохраним лучшую модель, чтобы в будущем ее могли взять и использовать, без обучения.

In [81]:
train_loop_with_scheduler(exp_model, X_train, y_train, X_test, y_test, config)

Epoch 0, accuracy = 0.7059999704360962
Epoch 1, accuracy = 0.7064999938011169
Epoch 2, accuracy = 0.7071999907493591
Epoch 3, accuracy = 0.7080000042915344
Epoch 4, accuracy = 0.708299994468689
Epoch 5, accuracy = 0.7089999914169312
Epoch 6, accuracy = 0.7098999619483948
Epoch 7, accuracy = 0.7102000117301941
Epoch 8, accuracy = 0.7111999988555908
Epoch 9, accuracy = 0.7112999558448792
Epoch 10, accuracy = 0.7112999558448792
Epoch 11, accuracy = 0.7111999988555908
Epoch 12, accuracy = 0.7111999988555908
Epoch 13, accuracy = 0.7110999822616577
Epoch 14, accuracy = 0.7112999558448792
Epoch 15, accuracy = 0.7111999988555908
Epoch 16, accuracy = 0.7112999558448792
Epoch 17, accuracy = 0.7111999988555908
Epoch 18, accuracy = 0.7113999724388123
Epoch 19, accuracy = 0.7110999822616577
Epoch 20, accuracy = 0.7110999822616577
Epoch 21, accuracy = 0.7111999988555908
Epoch 22, accuracy = 0.7112999558448792
Epoch 23, accuracy = 0.7110999822616577
Epoch 24, accuracy = 0.7110999822616577
Epoch 25, a

## Задание №9

Напишите код, который сохранит модель в файл `model.pt`.

In [29]:
# Впоследствии эту модель можно будет загрузить вот так:
model_loaded = ExpModel(num_classes=len(y_test.unique()))
model_loaded.load_state_dict(torch.load("model.pt"))